## upload to DB


In [1]:
import time
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_aws import BedrockEmbeddings

## Creating Index

In [ ]:
def create_index():
    pc = Pinecone(api_key="312601e0-eb28-418c-82f5-f3cd8c0d14c3")

    index_name = "langchain-rag-index"  # change if desired

    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    data_embeddings=BedrockEmbeddings(
    credentials_profile_name= 'default',
    model_id='amazon.titan-embed-text-v1',
    region_name='us-east-1')
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )
        while not pc.describe_index(index_name).status["ready"]:
            time.sleep(1)

    index = pc.Index(index_name)
    vector_store = PineconeVectorStore(index=index, embedding=data_embeddings)
    return vector_store

## Add Documents:

In [ ]:
from langchain_core.documents import Document
def add_documents(vector_store):
    document_1 = Document(page_content="""WFM enables you to configure multiple types of time off. Examples of time-off types include vacation,
sick leave, paid time off, holiday, and maternity leave.
By default, WFM creates the Vacation time-off type. This time-off type cannot be deleted and does
not belong exclusively to any site.
You can configure a time-off rule for a one or multiple time-off types. When you configure multiple
time-off types for the same rule, the time-off balance is calculated and accrued for all time-off types
associated with that rule.
Use the procedures in this topic to create, copy, edit, and delete time-off types, and associate/
disassociate time-off types with sites.
You assign time-off type/time-off rule combinations to agents by using the Configuration > Agents
> Time Off pane.""", metadata={"leave": "timeoff"})
    document_2 = Document(page_content="thud", metadata={"bar": "baz"})
    document_3 = Document(page_content="i will be deleted :(")

    documents = [document_1, document_2, document_3]
    ids = ["1", "2", "3"]
    vector_store.add_documents(documents=documents, ids=ids)

## Execute

vector_store = create_index()
add_documents(vector_store)